# Testing rotation of detector to change cake position in pyFAI

Download test image from: http://www.silx.org/pub/pyFAI/testimages/mock.tif
This test image is useful as it provides a clear indication of how the detector is aligned.

In [ ]:
import fabio
import xrdfit
from pyFAI.gui import jupyter
import pyFAI.detectors, pyFAI.azimuthalIntegrator
import math
import numpy as np
from xrdfit.spectrum_fitting import FitSpectrum

Load test image from file

In [ ]:
img = fabio.open(r"C:\Users\Peter\Documents\git\pyFAI-integration-caking\data\moke.tif").data
jupyter.display(img, label ="Fake diffraction image")

Set up mock detector - normally you would load a calibration here

In [ ]:
detector = pyFAI.detectors.Detector(pixel1=1e-4, pixel2=1e-4)
ai = pyFAI.azimuthalIntegrator.AzimuthalIntegrator(dist=0.1, detector=detector)
# Center the detector on the center of the pattern at 300, 300.
ai.setFit2D(100, 300, 300)

Now rotate the detector. `rot3` is the angle to rotate the detector. See [here](https://pyfai.readthedocs.io/en/latest/usage/tutorial/Geometry/geometry.html#Conclusion) for a schematic of the detector and rotations.

With 36 cakes and 100 slices the resolution is poor but xrdfit plotting is fast.
With 360 cakes and 1000 slices it is easier to see image reconstructed from the cakes but xrdfit plotting is slow. I think this is just a limitation of the radial plotting in matplotlib.

In [ ]:
num_cakes = 360
num_radial_slices = 1000

# Rotate the detector so that the cardinal direction is in the center of the
# first cake.
cake_angle = 360 / num_cakes
ai.rot3 = (cake_angle / 2 / 180) * math.pi

Display the caked image in a flat 2d perspective.

In [ ]:
res2d = ai.integrate2d(img, num_radial_slices, num_cakes, unit="2th_deg")
jupyter.plot2d(res2d, label="moke")

pyFAI cakes the data anticlockwise which is the opposite direction to DAWN. To replicate a DAWN caked output the order of the cakes in the intensity variable need to be flipped.

pyFAI also starts caking the data from the `East` direction (compared to DAWN which starts caking from the West) so the angle of the first cake is 270 degrees clockwise of the North direction.

By default, np.savetxt uses a space as a column delimiter. `xrdfit` defaults to tab spacing but this can be changed using the `delimiter` argument.

In [ ]:
# Flip the intensity data to order cakes clockwise rather than anticlockwise
intensity = np.flip(res2d.intensity.T, axis=1)

# Reshape radial labels to 2D array so they can be attached to the intensity data.
radial = np.reshape(res2d.radial, (-1, 1))

result = np.hstack((radial, intensity))
np.savetxt("example_caked_spectrum.txt", result)
spectrum = xrdfit.spectrum_fitting.FitSpectrum("example_caked_spectrum.txt", first_cake_angle=270, delimiter=" ")
spectrum.plot_polar()

The color change is not important - it is just a different matplotlib colormap to the `jupyter.plot2d` function.

An alternative to flipping the data is to use the new anticlockwise fit option of `xrdfit`. Note that the `first_cake_angle` has to be displaced by one cake width as we are plotting data the other way.

In [ ]:
# Just transpose rows/columns, dont flip.
intensity = res2d.intensity.T

# Reshape radial labels to 2D array so they can be attached to the intensity data.
radial = np.reshape(res2d.radial, (-1, 1))

result = np.hstack((radial, intensity))
np.savetxt("example_caked_spectrum.txt", result)
spectrum = xrdfit.spectrum_fitting.FitSpectrum("example_caked_spectrum.txt", first_cake_angle=270-cake_angle, delimiter=" ", cake_order="anticlockwise")
spectrum.plot_polar()